<a href="https://colab.research.google.com/github/Swastik200/30DaysOfDSA/blob/main/Genetic_Algorithm3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyMetaheuristic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 2.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from pyMetaheuristic.algorithm import genetic_algorithm
from pyMetaheuristic.utils import graphs

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp "/content/drive/MyDrive/CMAPSS.zip" "/content"
!unzip CMAPSS.zip

Archive:  CMAPSS.zip
  inflating: CMAPSS/readme.txt       
  inflating: CMAPSS/RUL_FD001.txt    
  inflating: CMAPSS/RUL_FD002.txt    
  inflating: CMAPSS/RUL_FD003.txt    
  inflating: CMAPSS/RUL_FD004.txt    
  inflating: CMAPSS/test_FD001.txt   
  inflating: CMAPSS/test_FD002.txt   
  inflating: CMAPSS/test_FD003.txt   
  inflating: CMAPSS/test_FD004.txt   
  inflating: CMAPSS/train_FD001.txt  
  inflating: CMAPSS/train_FD002.txt  
  inflating: CMAPSS/train_FD003.txt  
  inflating: CMAPSS/train_FD004.txt  
  inflating: CMAPSS/x.txt            


In [5]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['sensor_{}'.format(i) for i in range(1, 22)]
col_names = index_names + setting_names + sensor_names

In [6]:
path = '/content/CMAPSS/'
df_train = pd.read_csv(path + 'train_FD003.txt', sep='\s+', header=None, index_col=False, names=col_names)
df_test = pd.read_csv(path + 'test_FD003.txt', sep='\s+', header=None, index_col=False, names=col_names)
y_test = pd.read_csv(path + 'RUL_FD003.txt', sep='\s+', header=None, index_col=False, names=['RUL'])

In [7]:
# Calculate 'RUL' column based on 'time_cycles' and 'unit_number' columns
max_cycles = df_train.groupby('unit_number')['time_cycles'].max()
df_train['RUL'] = df_train.apply(lambda row: max_cycles[row['unit_number']] - row['time_cycles'], axis=1)
# Calculate 'RUL' column based on 'time_cycles' and 'unit_number' columns
max_cycles = df_test.groupby('unit_number')['time_cycles'].max()
df_test['RUL'] = df_test.apply(lambda row: max_cycles[row['unit_number']] - row['time_cycles'], axis=1)


In [8]:
train = df_train.copy()
test = df_test.copy()
display(train)
display(test)

,unit_number,time_cycles,setting_1,setting_2,setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL
0,1,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,2388.01,8145.32,8.4246,0.03,391,2388,100.0,39.11,23.3537,258.0
1,1,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,2388.03,8152.85,8.4403,0.03,392,2388,100.0,38.99,23.4491,257.0
2,1,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,2388.00,8150.17,8.3901,0.03,391,2388,100.0,38.85,23.3669,256.0
3,1,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,2388.08,8146.56,8.3878,0.03,392,2388,100.0,38.96,23.2951,255.0
4,1,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,2388.03,8147.80,8.3869,0.03,392,2388,100.0,39.14,23.4583,254.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24715,100,148,-0.0016,-0.0003,100.0,518.67,643.78,1596.01,1424.11,14.62,...,2388.30,8138.08,8.5036,0.03,394,2388,100.0,38.44,22.9631,4.0
24716,100,149,0.0034,-0.0003,100.0,518.67,643.29,1596.38,1429.14,14.62,...,2388.28,8144.36,8.5174,0.03,395,2388,100.0,38.50,22.9746,3.0
24717,100,150,-0.0016,0.0004,100.0,518.67,643.84,1604.53,1431.41,14.62,...,2388.24,8135.95,8.5223,0.03,396,2388,100.0,38.39,23.0682,2.0
24718,100,151,-0.0023,0.0004,100.0,518.67,643.94,1597.56,1426.57,14.62,...,2388.26,8141.24,8.5148,0.03,395,2388,100.0,38.31,23.0753,1.0


,unit_number,time_cycles,setting_1,setting_2,setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL
0,1,1,-0.0017,-0.0004,100.0,518.67,641.94,1581.93,1396.93,14.62,...,2387.94,8133.48,8.3760,0.03,391,2388,100.0,39.07,23.4468,232.0
1,1,2,0.0006,-0.0002,100.0,518.67,642.02,1584.86,1398.90,14.62,...,2388.01,8137.44,8.4062,0.03,391,2388,100.0,39.04,23.4807,231.0
2,1,3,0.0014,-0.0003,100.0,518.67,641.68,1581.78,1391.92,14.62,...,2387.94,8138.25,8.3553,0.03,391,2388,100.0,39.10,23.4244,230.0
3,1,4,0.0027,0.0001,100.0,518.67,642.20,1584.53,1395.34,14.62,...,2387.96,8137.07,8.3709,0.03,392,2388,100.0,38.97,23.4782,229.0
4,1,5,-0.0001,0.0001,100.0,518.67,642.46,1589.03,1395.86,14.62,...,2387.97,8134.20,8.4146,0.03,391,2388,100.0,39.09,23.3950,228.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16591,100,243,0.0011,-0.0003,100.0,518.67,643.04,1594.99,1411.28,14.62,...,2388.24,8162.24,8.2939,0.03,395,2388,100.0,39.40,23.4949,4.0
16592,100,244,-0.0024,0.0002,100.0,518.67,642.47,1591.27,1417.64,14.62,...,2388.26,8163.24,8.3247,0.03,394,2388,100.0,39.42,23.6011,3.0
16593,100,245,-0.0053,0.0002,100.0,518.67,642.70,1593.81,1412.70,14.62,...,2388.28,8162.12,8.3264,0.03,394,2388,100.0,39.43,23.5482,2.0
16594,100,246,-0.0006,0.0001,100.0,518.67,642.19,1595.63,1406.58,14.62,...,2388.33,8164.20,8.2908,0.03,395,2388,100.0,39.40,23.6687,1.0


In [9]:
X_train = df_train.drop(columns=['RUL'])
y_train = df_train['RUL']
X_test = df_test.drop(columns=['RUL'])
y_test = y_test['RUL']


In [10]:
parameters = {
    'population_size': 250,
    'min_values': [-5] * len(X_train.columns),  # Adjust according to your feature space
    'max_values': [5] * len(X_train.columns),  # Adjust according to your feature space
    'generations': 10,
    'mutation_rate': 0.1,
    'elite': 1,
    'eta': 1,
    'mu': 1,
    'verbose': True,
    'start_init': None,
    'target_value': None
}

In [11]:
def target_function(variables_values):
    # Train the regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Predict using the trained model
    predictions = model.predict(X_test)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    return rmse

In [12]:
# Assuming you have already loaded your dataset into variables df_train and df_test

# Define X and y for the training dataset
X_train = df_train.drop(columns=['RUL'])  # Features
y_train = df_train['RUL']  # Target variable

# Define X and y for the testing dataset
X_test = df_test.drop(columns=['RUL'])  # Features
y_test = df_test['RUL']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [13]:
ga_result = genetic_algorithm(target_function=target_function, **parameters)

Generation =  0  f(x) =  60.61804665033728
Generation =  1  f(x) =  60.61804665033728
Generation =  2  f(x) =  60.61804665033728
Generation =  3  f(x) =  60.61804665033728
Generation =  4  f(x) =  60.61804665033728
Generation =  5  f(x) =  60.61804665033728
Generation =  6  f(x) =  60.61804665033728
Generation =  7  f(x) =  60.61804665033728
Generation =  8  f(x) =  60.61804665033728
Generation =  9  f(x) =  60.61804665033728
Generation =  10  f(x) =  60.61804665033728


In [14]:
optimized_model = LinearRegression().fit(X_test, y_test)

In [15]:
rmse_test = np.sqrt(mean_squared_error(y_test, optimized_model.predict(X_test)))
r2_test = r2_score(y_test, optimized_model.predict(X_test))
mae_test = mean_absolute_error(y_test, optimized_model.predict(X_test))

In [16]:
print('RMSE (Test):', rmse_test)
print('R2 Score (Test):', r2_test)
print('MAE (Test):', mae_test)

RMSE (Test): 60.416296816871835
R2 Score (Test): 0.6275612711793261
MAE (Test): 44.74673215008091
